In [1]:
%load_ext autoreload

In [2]:
import os
print( os.getcwd() )
os.chdir( "/var/genie-in-the-box/src" )
print( os.getcwd() )

/var/genie-in-the-box/src/notebooks
/var/genie-in-the-box/src


In [4]:
%autoreload

import sys
import os
import json

# os.environ[ "LANGCHAIN_WANDB_TRACING" ] = "true"
# # wandb documentation to configure wandb using env variables
# # https://docs.wandb.ai/guides/track/advanced/environment-variables
# # here we are configuring the wandb project name
# os.environ[ "WANDB_PROJECT" ] = "deepily-dataframe-agent"

# path = "/var/genie-in-the-box/src/lib"
# if path not in sys.path:
#     sys.path.append( path )
# else:
#     print( f"[{path}] already in sys.path" )
# 
# print( sys.path )

import lib.util             as du

import lib.util_pandas      as up
import lib.util_stopwatch   as sw
import lib.util_code_runner as ucr

from solution_snapshot_mgr  import SolutionSnapshotManager
from agent_refactoring      import RefactoringAgent
from agent_function_mapping import FunctionMappingAgent
from agent_calendaring      import CalendaringAgent

import openai
import pandas as pd

debug   = True
verbose = True



/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug
/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev
/usr/local/lib/python3.10
/usr/local/lib/python3.10/lib-dynload
/usr/local/lib/python3.10/site-packages
/usr/local/lib/python310.zip
/var/genie-in-the-box/src/notebooks


In [5]:
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df[ ( df.event_type == 'concert' ) ].start_date.value_counts()


start_date
2023-11-01    2
2023-10-28    2
2023-09-30    2
2023-10-13    1
2023-10-30    1
2023-10-22    1
2023-10-21    1
2023-10-20    1
2023-10-19    1
2023-10-15    1
2023-09-01    1
2023-09-06    1
2023-10-03    1
2023-09-28    1
2023-09-27    1
2023-09-20    1
2023-09-12    1
2023-09-09    1
2023-10-07    1
Name: count, dtype: int64

In [6]:
foo = """[{ "name": "get_this_weeks_events_by_type", "description": "Gets events of a certain type that occur within a certain date range (default is this week)", "parameters": { "type": "object", "properties": { "df": { "type": "object", "description": "The dataframe containing the events" }, "event_type": { "type": "string", "description": "The type of event to filter for" } }, "required": ["df"] } }]"""

def get_parameter_names( signature_dict ):
    
    params = []
    for k, v in signature_dict[ 0 ][ "parameters" ][ "properties" ].items():
        if k != "df":
            params.append( k )
            print( k, v )
        else:
            print( "skipping df" )
            
    return params
    
foo_dict = json.loads( foo )
get_parameter_names( foo_dict )

skipping df
event_type {'type': 'string', 'description': 'The type of event to filter for'}


['event_type']

In [7]:
import re
import json

# Original string
original_string = "{ 'name': 'get_this_weeks_events_by_type', 'description': 'Gets events of a certain type that occur within the current week', 'parameters': { 'type': 'object', 'properties': { 'df': { 'type': 'object', 'description': 'The dataframe containing the events' }, 'event_type': { 'type': 'string', 'description': 'The type of event to filter for' } }, 'required': ['df', 'event_type'] } }"

# Regular expression to match single quotes at the start or end of words
pattern = r"'(?=\w)|(?<=\w)'"

# Replace matched patterns with double quotes
fixed_string = re.sub( pattern, '"', original_string )
print( fixed_string )

# Encode the string as a JSON object
json_object = json.loads( fixed_string )

# Print the JSON object
# print(json_object)

{ "name": "get_this_weeks_events_by_type", "description": "Gets events of a certain type that occur within the current week", "parameters": { "type": "object", "properties": { "df": { "type": "object", "description": "The dataframe containing the events" }, "event_type": { "type": "string", "description": "The type of event to filter for" } }, "required": ["df", "event_type"] } }


### Prototype & test new function prompt

In [8]:
%autoreload
fm_agent      = FunctionMappingAgent( "/src/conf/long-term-memory/events.csv", debug=True )
results_dict  = fm_agent.run_prompt( question="Is Grandma's birthday in October?" )
results_dict

Token count for `system_message`: [876]
Token count for `user_message`: [16]
Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 15,837 ms


{'thoughts': "The question is asking about a specific event, a birthday, for a specific person, grandma. This maps to the function 'get_birthday_by_name' which retrieves the birthday of a given person from a dataframe. The kwargs key would be 'name' and the value would be 'grandma'.",
 'is_event_function_call': True,
 'question': 'is grandmas birthday in october',
 'function_name': 'get_birthday_by_name',
 'kwargs_key': 'name',
 'kwargs_value': 'grandma',
 'import_as': 'import lib.autogen.util_calendaring_2 as uc2',
 'call_template': "uc2.get_birthday_by_name( df, name='grandma' )",
 'code': ['import lib.autogen.util_calendaring_2 as uc2',
  "uc2.get_birthday_by_name( df, name='grandma' )"]}

In [10]:
%autoreload
if fm_agent.is_runnable():
    
    code_response = fm_agent.run_code()
    code_response


last command, before [uc2.get_birthday_by_name( df, name='grandma' )]:
return_type [string]
ERROR: return_type [string] and last command [uc2.get_birthday_by_name( df, name='grandma' )] are incompatible!
last command,  after [uc2.get_birthday_by_name( df, name='grandma' )]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_2 as uc2
uc2.get_birthday_by_name( df, name='grandma' )

Executing /var/genie-in-the-box/io/code.py... Done!


In [12]:
fm_agent.code_response_dict

{'return_code': 0, 'output': 'No results returned'}

In [13]:
fm_agent.format_output()

Token count for `formatting preamble`: [59]
Token count for `formatting instructions`: [51]
Asking ChatGPT [gpt-3.5-turbo-0613]...
Asking ChatGPT [gpt-3.5-turbo-0613]... in 805 ms


'No results returned.'

In [14]:
fm_agent.signatures_dict.keys()

dict_keys(['get_birthday_by_name', 'get_todays_events_by_type', 'get_this_weeks_events_by_type'])

### Test new refactoring agent

In [18]:
%autoreload
    
def begin_refactoring_similar_questions( question, run_prompt=False, threshold=90.0, debug=False ):
    
    path_to_snapshots = du.get_project_root() + "/src/conf/long-term-memory/solutions/"
    snapshot_mgr      = SolutionSnapshotManager( path_to_snapshots, debug=debug )
    exemplar_snapshot = snapshot_mgr.get_snapshots_by_question( question )[ 0 ][ 1 ]
    similar_snapshots = snapshot_mgr.get_snapshots_by_code_similarity( exemplar_snapshot, threshold=threshold )
        
    agent = RefactoringAgent( 
        similar_snapshots=similar_snapshots, 
        path_to_solutions="/src/conf/long-term-memory/solutions", 
        debug=True, verbose=True 
    )
    
    if run_prompt and agent.is_promptable():
        agent.run_prompt()
    else:
        print( f"run_prompt={run_prompt} -- OR -- No similar snapshots found for [{du.truncate_string( exemplar_snapshot.question, max_len=32 )}]" )
        
    return agent
    
# agent = begin_refactoring_similar_questions( "What birthdays do I have this week?", threshold=90.0, debug=False )        

In [19]:
%autoreload
rf_agent = begin_refactoring_similar_questions( "When is Juan's birthday?", run_prompt=False )
# agent


------------------------------------------------------------------------------------------------------------------------
- Found [42] synonymous questions
------------------------------------------------------------------------------------------------------------------------

Synonymous question [when is juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [when are we celebrating juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [when is juans birthday party] for snapshot.question [when is juans birthday]
Synonymous question [is juans birthday on october th] for snapshot.question [when is juans birthday]
Synonymous question [what is juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [is juans birthday in october] for snapshot.question [when is juans birthday]
Synonymous question [what day is today] for snapshot.question [what day is today]
Synonymous question [whats todays date] for snapsho

In [20]:
%autoreload
rf_agent.run_prompt()


------------------------------------------------------------------------------------------------------------------------
- Token count for `system_message`: [778]
------------------------------------------------------------------------------------------------------------------------


        I'm going to show you 4 Python code snippets that are similar, along with the questions they were created to answer.
        How would you coalesce or refactor them so that you only need to call one function in all 4 scenarios?
        How would you name the function in a way that clearly explains exactly what the date range is, e.g., 'get_this_months_events_by_type', as well as what the function does?
        Descriptive function names MUST look like: `get_birthday_by_name`, or `get_todays_events`, `get_todays_events_by_type`, `get_tomorrows_events`, `get_this_weeks_events`, `get_this_weeks_events_by_type`,`get_this_months_events_by_type`, etc.,

        As you generate the Python 3.10 code need

{'thoughts': "The four code snippets are essentially doing the same thing: they are retrieving the birthday of a person by their name from a dataframe. The only difference between them is the name of the person. Therefore, we can refactor these snippets into a single function that takes the name as a parameter. If the query does not return a result, we can return a message indicating that the person's birthday is not found in the dataframe.",
 'code': ['import pandas as pd',
  'def get_birthday_by_name(df: pd.DataFrame, **kwargs):',
  "    name = kwargs.get('name')",
  '    if name is not None:',
  "        result = df[df['name'] == name]['birthday']",
  '        if not result.empty:',
  '            return result.values[0]',
  "    return 'Birthday not found'"],
 'function_name': 'get_birthday_by_name',
 'parameters': 'df: a pandas DataFrame containing the data, **kwargs: a dictionary containing the name of the person whose birthday is to be retrieved',
 'gpt_function_signatures': "[{

In [19]:
%autoreload
rf_agent.refactor_code( update_example_code=True, debug=False )

Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_2.py]... Done!
Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_2.json]... Done!
Writing file [/var/genie-in-the-box/io/lib/autogen/util_calendaring_2.py]... Done!

Generating embedding for [import lib.autogen.util_calendaring_2 as uc2 print(uc2.get_birth...]...
Generating embedding for [import lib.autogen.util_calendaring_2 as uc2 print(uc2.get_birth...]... Done! in 235 ms

solution_file value provided: [when-is-juans-birthday-0.json]...
File path: /var/genie-in-the-box/src/conf/long-term-memory/solutions/when-is-juans-birthday-0.json

Generating embedding for [import lib.autogen.util_calendaring_2 as uc2 print(uc2.get_birth...]...
Generating embedding for [import lib.autogen.util_calendaring_2 as uc2 print(uc2.get_birth...]... Done! in 215 ms

solution_file value provided: [when-is-jimmys-birthday-0.json]...
File path: /var/genie-in-the-box/src/conf/long-term-memory/solutions/when-is-jimmys-birth

In [20]:
rf_agent.run_code()

last command, before [print(uc2.get_birthday_by_name(df, name='Juan'))]:
return_type [string]
ERROR: return_type [string] and last command [print(uc2.get_birthday_by_name(df, name='Juan'))] are incompatible!
last command,  after [print(uc2.get_birthday_by_name(df, name='Juan'))]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_2 as uc2
print(uc2.get_birthday_by_name(df, name='Juan'))

Executing /var/genie-in-the-box/io/code.py... Done!

------------------------------------------------------------------------------------------------------------------------
- Code output
------------------------------------------------------------------------------------------------------------------------

2023-10-13T00:00:00.000000000

last command, before [print(uc2.get_birthday_by_name(df, name='Jimm

[{'return_code': 0, 'output': '2023-10-13T00:00:00.000000000'},
 {'return_code': 0, 'output': 'No birthday found for Jimmy'},
 {'return_code': 0, 'output': 'No birthday found for Bobby'},
 {'return_code': 0, 'output': 'No birthday found for Anita'}]

### Re-factor, test and debug calendaring agent

In [27]:
%autoreload
agent = CalendaringAgent( path_to_df="/src/conf/long-term-memory/events.csv", debug=True, verbose=True )
    
# question         = "What todo items do I have on my calendar for this week?"
# question         = "What todo items do I have on my calendar for today?"
# question         = "Do I have any birthdays on my calendar this week?"
# question         = "When is Juan's birthday?"
question         = "When is Jimmy's birthday?"
timer            = sw.Stopwatch( msg=f"Processing [{question}]..." )
response_dict    = agent.run_prompt( question )
code_response    = agent.run_code()
# formatted_output = agent.format_output()
timer.print( use_millis=True )



Processing [When is Jimmy's birthday?]...

------------------------------------------------------------------------------------------------------------------------
- Token count for `system_message`: [888]
------------------------------------------------------------------------------------------------------------------------


        You are an expert software engineer working with a pandas dataframe in Python containing calendaring and events information. The name of the dataframe is `df`.

        This is the ouput from `print(df.head().to_csv())`, in CSV format:

        start_date,end_date,start_time,end_time,event_type,recurrent,recurrence_interval,priority_level,name,relationship,description_who_what_where
2023-09-01,2023-09-04,00:00,23:59,concert,False,,none,Jenny,coworker,Concert of Jenny at the city center
2023-09-01,2023-09-01,05:25,17:22,todo,False,,highest,Gregorio,friend,Send out invitations for the party for Gregorio
2023-09-01,2023-09-01,13:27,01:59,appointment,False,,h

In [30]:
%autoreload

formatted_output = agent.format_output()
du.print_banner( question, prepend_nl=True )

for line in formatted_output.split( "\n" ):
    print( line )


------------------------------------------------------------------------------------------------------------------------
- Token count for `formatting preamble`: [60]
------------------------------------------------------------------------------------------------------------------------


            You are an expert in converting raw data into conversational English.

            The output is the result of a query on a pandas dataframe about events on my calendar.

            The query is: `When is Jimmy's birthday?`

            The output is: `No birthday information for Jimmy`
            

------------------------------------------------------------------------------------------------------------------------
- Token count for `formatting instructions`: [50]
------------------------------------------------------------------------------------------------------------------------


        Reformat and rephrase the data that I just showed you in conversational English so that it a